# Project 2 Machine Learning

In [ ]:
%load_ext autoreload
%autoreload 2
%reload_ext lab_black
%matplotlib inline

In [ ]:
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

## Import libraries
Use tensorflow.keras or Keras

In [ ]:
import numpy as np
import scipy as scipy
import os
import pickle
import matplotlib
import matplotlib.pyplot as plt
import cv2

from tensorflow.keras import Model
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.applications import *
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler

from datetime import datetime
from helpers import *

In [ ]:
# Import all the necessary for our model
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, TensorBoard
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import (
    Activation,
    Dropout,
    Flatten,
    Dense,
    Conv2D,
    MaxPooling2D,
    LeakyReLU,
    GaussianNoise,
)
from tensorflow.keras import backend as K
from kerassurgeon.operations import delete_layer, insert_layer, replace_layer
from keras.callbacks import Callback
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score
from tensorflow.compat.v2.keras.layers import BatchNormalization

In [ ]:
baseline = 0
best = 1

## Loading images

In [ ]:
image_dir_train = "data/training/images/"
files = os.listdir(image_dir_train)
n_train = len(files)
print(f"Loading training images, images loaded: {n_train} ")
imgs_train = np.asarray(
    [load_image(image_dir_train + files[i]) for i in range(n_train)]
)
gt_dir_train = "data/training/groundtruth/"
print(f"Loading groundtruth images, images loaded: {n_train} ")
gt_imgs_train = np.asarray(
    [load_image(gt_dir_train + files[i]) for i in range(n_train)]
)

In [ ]:
image_size = 400
# Patches for training
img_patches_train = [
    crop_image(imgs_train[i], image_size, image_size) for i in range(n_train)
]
gt_patches_train = [
    crop_image(gt_imgs_train[i], image_size, image_size) for i in range(n_train)
]

# Separate features and labels
X_train = np.asarray(
    [
        img_patches_train[i][j]
        for i in range(len(img_patches_train))
        for j in range(len(img_patches_train[i]))
    ]
)
Y_train = np.asarray(
    [
        gt_patches_train[i][j]
        for i in range(len(gt_patches_train))
        for j in range(len(gt_patches_train[i]))
    ]
)

In [ ]:
image_dir_val = "data/validating/images/"
files = os.listdir(image_dir_val)
n_val = len(files)
print(f"Loading validating images, images loaded: {n_val} ")
imgs_val = np.asarray([load_image(image_dir_val + files[i]) for i in range(n_val)])
gt_dir_val = "data/validating/groundtruth/"
print(f"Loading validating groundtruth, images loaded: {n_val} ")
gt_imgs_val = np.asarray([load_image(gt_dir_val + files[i]) for i in range(n_val)])

In [ ]:
image_size = 400
# Patches for validating
img_patches_val = [
    crop_image(imgs_val[i], image_size, image_size) for i in range(n_val)
]
gt_patches_val = [
    crop_image(gt_imgs_val[i], image_size, image_size) for i in range(n_val)
]

# Separate features and labels
X_val = np.asarray(
    [
        img_patches_val[i][j]
        for i in range(len(img_patches_val))
        for j in range(len(img_patches_val[i]))
    ]
)
Y_val = np.asarray(
    [
        gt_patches_val[i][j]
        for i in range(len(gt_patches_val))
        for j in range(len(gt_patches_val[i]))
    ]
)

## Select data augmentation
Method 1 or 2:

In [ ]:
method = 1
data_aug_factor = 1
if method == 2:
    data_aug_factor = 9

In [ ]:
if method == 2:

    def pad_matrix(mat, h_pad, w_pad, val=0):
        h_pad = int(h_pad)
        w_pad = int(w_pad)
        if len(mat.shape) == 3:
            padded_mat = np.pad(
                mat,
                ((h_pad, h_pad), (w_pad, w_pad), (0, 0)),
                mode="constant",
                constant_values=((val, val), (val, val), (0, 0)),
            )
        elif len(mat.shape) == 2:
            padded_mat = np.pad(
                mat,
                ((h_pad, h_pad), (w_pad, w_pad)),
                mode="constant",
                constant_values=((val, val), (val, val)),
            )
        else:
            raise ValueError("This method can only handle 2d or 3d arrays")
        return padded_mat

    def imag_rotation(X, Y, number_rotations=8):

        w = X.shape[1]
        w_2 = w // 2  # half of the width
        padding = 82
        Xrs = X
        Yrs = Y

        Xrs = np.expand_dims(Xrs, 0)
        Yrs = np.expand_dims(Yrs, 0)

        thetas = np.random.randint(0, high=360, size=number_rotations)
        for theta in thetas:
            Xr = pad_matrix(
                X, padding, padding
            )  # Selected for the specific case of images of (400,400)
            Yr = pad_matrix(
                Y, padding, padding
            )  # Selected for the specific case of images of (400,400)
            Xr = scipy.ndimage.rotate(Xr, theta, reshape=False)
            Yr = scipy.ndimage.rotate(Yr, theta, reshape=False)
            theta = theta * np.pi / 180
            a = int(
                w_2 / (np.sqrt(2) * np.cos(np.pi / 4 - np.mod(theta, np.pi / 2)))
            )  # width and height of the biggest square inside the rotated square
            w_p = w_2 + padding
            Xr = Xr[w_p - a : w_p + a, w_p - a : w_p + a, :]
            Yr = Yr[w_p - a : w_p + a, w_p - a : w_p + a]

            Xr = cv2.resize(Xr, dsize=(w_2 * 2, w_2 * 2), interpolation=cv2.INTER_CUBIC)
            Yr = cv2.resize(Yr, dsize=(w_2 * 2, w_2 * 2), interpolation=cv2.INTER_CUBIC)

            if np.random.choice(2) == 1:
                Xr = np.flipud(Xr)
                Yr = np.flipud(Yr)

            if np.random.choice(2) == 1:
                Xr = np.fliplr(Xr)
                Yr = np.fliplr(Yr)

            Xr = np.expand_dims(Xr, 0)
            Yr = np.expand_dims(Yr, 0)
            Xrs = np.append(Xrs, Xr, axis=0)
            Yrs = np.append(Yrs, Yr, axis=0)

        return Xrs, Yrs

    def imag_rotation_aug(Xr, Yr, number_rotations=8):

        Xrs, Yrs = imag_rotation(Xr[0], Yr[0])
        for i in range(1, len(Xr)):
            Xrr, Yrr = imag_rotation(Xr[i], Yr[i])
            Xrs = np.append(Xrs, Xrr, axis=0)
            Yrs = np.append(Yrs, Yrr, axis=0)

        Xrs_shuf = []
        Yrs_shuf = []
        index_shuf = list(range(len(Xrs)))
        np.random.shuffle(index_shuf)
        for i in index_shuf:
            Xrs_shuf.append(Xrs[i])
            Yrs_shuf.append(Yrs[i])

        return Xrs_shuf, Yrs_shuf

In [ ]:
if method == 2:
    X_train, Y_train = imag_rotation_aug(imgs_train, gt_imgs_train)

In [ ]:
if method == 2:
    X_val, Y_val = imag_rotation_aug(imgs_val, gt_imgs_val)

## Generating mini-batch and running data augmentation

In [ ]:
if method == 2:

    def create_minibatch(X, Y, n):

        # Fix the seed
        # np.random.seed(1)

        # We define the window size of 72, batch size of 100 (empirically chosen)
        # and patch size should correspond to 16
        w_size = 64
        batch_size = 100
        patch_size = 16
        num_images = n

        while True:
            # Generate one minibatch
            batch_image = np.empty((batch_size, w_size, w_size, 3))
            batch_label = np.empty((batch_size, 2))

            for i in range(batch_size):

                # Select a random index representing an image
                random_index = np.random.choice(num_images)

                # Width of original image
                width = 400

                # Sample a random window from the image
                random_sample = np.random.randint(w_size // 2, width - w_size // 2, 2)

                # Create a sub image of size 72x72
                sampled_image = X[random_index][
                    random_sample[0] - w_size // 2 : random_sample[0] + w_size // 2,
                    random_sample[1] - w_size // 2 : random_sample[1] + w_size // 2,
                ]

                # Take its corresponding ground-truth image
                correspond_ground_truth = Y[random_index][
                    random_sample[0]
                    - patch_size // 2 : random_sample[0]
                    + patch_size // 2,
                    random_sample[1]
                    - patch_size // 2 : random_sample[1]
                    + patch_size // 2,
                ]

                # We set in the label depending on the threshold of 0.25
                # The label becomes either 0 or 1 by applying to_categorical with parameter 2
                label = to_categorical(
                    (np.array([np.mean(correspond_ground_truth)]) > 0.25) * 1, 2
                )

                # We put in the sub image and its corresponding label before yielding it
                batch_image[i] = sampled_image
                batch_label[i] = label

            # Yield the mini_batch to the model
            yield (batch_image, batch_label)

In [ ]:
if method == 1:

    def create_minibatch(X, Y, n, batch_size=100):

        # Fix the seed
        np.random.seed(1)

        # We define the window size of 72, batch size of 100 (empirically chosen)
        # and patch size should correspond to 16
        w_size = 64
        batch_size = batch_size
        patch_size = 16
        num_images = n

        while True:
            # Generate one minibatch
            batch_image = np.empty((batch_size, w_size, w_size, 3))
            batch_label = np.empty((batch_size, 2))

            for i in range(batch_size):

                # Select a random index represnting an image
                random_index = np.random.choice(num_images)

                # Width of original image
                width = 400

                # Sample a random window from the image
                random_sample = np.random.randint(w_size // 2, width - w_size // 2, 2)

                # Create a sub image of size 72x72
                sampled_image = X[random_index][
                    random_sample[0] - w_size // 2 : random_sample[0] + w_size // 2,
                    random_sample[1] - w_size // 2 : random_sample[1] + w_size // 2,
                ]

                # Take its corresponding ground-truth image
                correspond_ground_truth = Y[random_index][
                    random_sample[0]
                    - patch_size // 2 : random_sample[0]
                    + patch_size // 2,
                    random_sample[1]
                    - patch_size // 2 : random_sample[1]
                    + patch_size // 2,
                ]

                # We set in the label depending on the threshold of 0.25
                # The label becomes either 0 or 1 by applying to_categorical with parameter 2
                label = to_categorical(
                    (np.array([np.mean(correspond_ground_truth)]) > 0.25) * 1, 2
                )

                # The image augmentation is based on both flipping and rotating (randomly in steps of 45°)
                # Random vertical and horizontal flip
                if np.random.choice(2) == 1:
                    sampled_image = np.flipud(sampled_image)

                if np.random.choice(2) == 1:
                    sampled_image = np.fliplr(sampled_image)

                # Random rotation in steps of 45°
                rotations = [0, 45, 90, 135, 180, 225, 270, 315]

                # We select a rotation degree randomly
                rotation_choice = np.random.choice(len(rotations))

                # Rotate it using the random value (uses the scipy library)
                sampled_image = scipy.ndimage.rotate(
                    sampled_image,
                    rotations[rotation_choice],
                    order=1,
                    reshape=False,
                    mode="reflect",
                )

                # We put in the sub image and its corresponding label before yielding it
                batch_image[i] = sampled_image
                batch_label[i] = label

            # Yield the mini_batch to the model
            yield (batch_image, batch_label)

In [ ]:
def precision(y_true, y_pred):
    """Precision metric.

    Only computes a batch-wise average of precision.

    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


def recall(y_true, y_pred):
    """Recall metric.

    Only computes a batch-wise average of recall.

    Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall


def f1(y_true, y_pred):
    prec = precision(y_true, y_pred)
    rec = recall(y_true, y_pred)
    return 2 * ((prec * rec) / (prec + rec + K.epsilon()))

## Resnet-Unet model Best

In [ ]:
if best:

    class resnet_unet_model:

        # Initialize the class
        def __init__(
            self,
            shape,
            batch_normalization,
            activation,
            dropout,
            amsgrad=False,
            lr=1e-4,
            noise=0.0,
        ):
            self.shape = shape
            self.batch_normalization = batch_normalization
            self.activation = activation
            self.dropout = dropout
            self.amsgrad = amsgrad
            self.lr = lr
            self.noise = noise
            self.model = self.initialize_resnet_unet_model()

        def conv_act(self, inputs, out_filters, activation="relu"):
            return Conv2D(
                filters=out_filters,
                activation=activation,
                kernel_size=3,
                strides=1,
                padding="same",
            )(inputs)

        def decoder(
            self,
            inputs,
            mid_filters=512,
            out_filters=256,
            activation="relu",
            block_name="decoder",
        ):
            with K.name_scope(block_name):
                if activation == "leaky_relu":
                    activation = None
                    conv = LeakyReLU(alpha=0.3)(
                        self.conv_act(inputs, mid_filters, activation)
                    )
                else:
                    conv = self.conv_act(inputs, mid_filters, activation)
                conv_tr = Conv2DTranspose(
                    filters=out_filters,
                    activation=activation,
                    kernel_size=4,
                    strides=2,
                    padding="same",
                )(conv)
            return conv_tr

        def initialize_resnet_unet_model(self):
            #         print(activation)

            # INPUT
            # shape     - Size of the input images
            # OUTPUT
            # model    - Compiled CNN

            # Define parameters
            max_pooling_size = 2
            max_pooling_strd = 2

            # Load a pretrained ResNet
            num_classes = 2
            resnet50 = ResNet50(
                include_top=False,
                weights="imagenet",
                classes=num_classes,
                input_shape=self.shape,
            )
            resnet50.compile(
                optimizer=Adam(lr=self.lr, amsgrad=self.amsgrad),
                loss="binary_crossentropy",
            )

            # ResNet convolution outputs
            conv5_out = resnet50.get_layer("conv5_block3_out").output
            conv4_out = resnet50.get_layer("conv4_block6_out").output
            conv3_out = resnet50.get_layer("conv3_block4_out").output
            conv2_out = resnet50.get_layer("conv2_block3_out").output

            pool = MaxPooling2D(
                max_pooling_size, strides=max_pooling_strd, padding="same"
            )(resnet50.get_output_at(0))
            dec_center = self.decoder(
                pool,
                mid_filters=self.shape[0] * 2,
                out_filters=self.shape[0],
                activation=self.activation,
                block_name="decoder_center",
            )

            if self.batch_normalization:
                dec_center = BatchNormalization()(dec_center)
            if self.dropout > 0:
                dec_center = Dropout(dropout)(dec_center)

            cat1 = Concatenate()([dec_center, conv5_out])
            dec5 = self.decoder(
                cat1,
                mid_filters=int(self.shape[0] * 2),
                out_filters=int(self.shape[0]),
                activation=self.activation,
                block_name="decoder5",
            )
            if self.batch_normalization:
                dec5 = BatchNormalization()(dec5)
            if self.dropout > 0:
                dec5 = Dropout(self.dropout)(dec5)

            cat2 = Concatenate()([dec5, conv4_out])
            dec4 = self.decoder(
                cat2,
                mid_filters=int(self.shape[0] * 2),
                out_filters=int(self.shape[0]),
                activation=self.activation,
                block_name="decoder4",
            )
            if self.batch_normalization:
                dec4 = BatchNormalization()(dec4)
            if self.dropout > 0:
                dec4 = Dropout(self.dropout)(dec4)

            cat3 = Concatenate()([dec4, conv3_out])
            dec3 = self.decoder(
                cat3,
                mid_filters=int(self.shape[0]),
                out_filters=int(self.shape[0] // 4),
                activation=self.activation,
                block_name="decoder3",
            )
            if self.batch_normalization:
                dec3 = BatchNormalization()(dec3)
            if self.dropout > 0:
                dec3 = Dropout(self.dropout)(dec3)

            cat2 = Concatenate()([dec3, conv2_out])
            dec2 = self.decoder(
                cat2,
                mid_filters=int(self.shape[0] // 2),
                out_filters=int(self.shape[0] // 2),
                activation=self.activation,
                block_name="decoder2",
            )
            if self.batch_normalization:
                dec2 = BatchNormalization()(dec2)
            if dropout > 0:
                dec2 = Dropout(self.dropout)(dec2)

            dec1 = self.decoder(
                dec2,
                mid_filters=int(self.shape[0] // 2),
                out_filters=int(self.shape[0] // 8),
                activation=self.activation,
                block_name="decoder1",
            )
            if self.batch_normalization:
                dec1 = BatchNormalization()(dec1)
            if self.dropout > 0:
                dec1 = Dropout(self.dropout)(dec1)

            dec0 = self.conv_act(dec1, out_filters=int(self.shape[0] // 8))
            conv_f = Conv2D(1, 1, activation="sigmoid", padding="same")(dec0)
            flatten_0 = Flatten()(conv_f)
            dense_0 = Dense(
                self.shape[0] / 2,
                kernel_regularizer=l2(1e-6),
                activity_regularizer=l2(1e-6),
            )(flatten_0)
            dropout_0 = Dropout(0.5)(dense_0)
            lk_relu_0 = LeakyReLU(alpha=0.1)(dropout_0)
            dense_1 = Dense(
                2, kernel_regularizer=l2(1e-6), activity_regularizer=l2(1e-6)
            )(lk_relu_0)
            dropout_1 = Dropout(0.2)(dense_1)
            output = Activation("sigmoid")(dropout_1)
            model = Model(inputs=resnet50.get_input_at(0), outputs=output)

            #           Compile the model using the binary crossentropy loss and the Adam optimizer for it
            #           We used the accuracy as a metric, but F1 score is also a plausible choice
            model.compile(
                loss="binary_crossentropy",
                optimizer=Adam(lr=lr, amsgrad=self.amsgrad),
                metrics=["accuracy", recall, f1],
            )

            # Print a summary of the model to see what has been generated
            model.summary()

            return model

        def train(
            self,
            epochs,
            steps_per_epoch,
            n_train=85,
            n_val=15,
            batch_size=100,
            data_aug_factor=1,
        ):

            # Early stopping callback after 10 steps
            early_stopping = EarlyStopping(
                monitor="val_loss", min_delta=0, patience=10, verbose=1, mode="auto"
            )

            # Reduce learning rate on plateau after 4 steps
            lr_callback = ReduceLROnPlateau(
                monitor="loss", factor=0.5, patience=4, verbose=1, mode="auto"
            )

            # Save best model
            weights_filename = "model_"
            if self.batch_normalization:
                weights_filename = weights_filename + "batch_"
            weights_filename = (
                weights_filename
                + self.activation
                + "_"
                + str(epochs)
                + "_"
                + "dropout_"
                + str(self.dropout)
                + "_"
                + "{epoch:03d}_"
                + "{f1:03f}_"
                + "{val_accuracy:03f}.h5"
            )
            save_best = ModelCheckpoint(
                weights_filename,
                save_best_only=True,
                monitor="val_loss",
                mode="auto",
                verbose=1,
            )

            # Place the callbacks in a list to be used when training
            #         callbacks = [cb, early_stopping, lr_callback]
            callbacks = [save_best, lr_callback]

            # Train the model using the previously defined functions and callbacks
            history = self.model.fit_generator(
                create_minibatch(
                    X_train, Y_train, data_aug_factor * n_train, batch_size=batch_size
                ),
                steps_per_epoch=steps_per_epoch,
                epochs=epochs,
                use_multiprocessing=False,
                workers=1,
                callbacks=callbacks,
                verbose=1,
                validation_data=create_minibatch(
                    X_val, Y_val, data_aug_factor * n_val, batch_size=batch_size
                ),
                validation_steps=steps_per_epoch,
            )

            return history

        def classify(self, X):
            # Subdivide the images into blocks with a stride and patch_size of 16
            img_patches = create_patches(X, 16, 16, padding=24)

            # Predict
            predictions = self.model.predict(img_patches)
            predictions = (predictions[:, 0] < predictions[:, 1]) * 1

            # Regroup patches into images
            #         return group_patches(predictions, X.self.shape[0])
            return group_patches(predictions, X.shape[0])

        def load(self, filename):
            # Load the model (used for submission)
            dependencies = {"recall": recall, "f1": f1}
            self.model = load_model(filename, custom_objects=dependencies)

        def save(self, filename):
            # Save the model (used to then load to submit)
            self.model.save(filename)

## Resnet-Unet model

In [ ]:
# if not baseline:
if not best:

    class resnet_unet_model:

        # Initialize the class
        def __init__(
            self,
            shape,
            batch_normalization,
            activation,
            dropout,
            amsgrad=False,
            lr=1e-4,
            noise=0.0,
        ):
            self.shape = shape
            self.batch_normalization = batch_normalization
            self.activation = activation
            self.dropout = dropout
            self.amsgrad = amsgrad
            self.lr = lr
            self.noise = noise
            self.model = self.initialize_resnet_unet_model()

        def conv_act(
            self, inputs, out_filters, activation="relu", batch_normalization=False
        ):
            conv = Conv2D(
                filters=out_filters, kernel_size=3, strides=1, padding="same"
            )(inputs)
            if batch_normalization:
                conv = BatchNormalization()(conv)
            return Activation(activation)(conv)

        def decoder(
            self,
            inputs,
            mid_filters=512,
            out_filters=256,
            activation="relu",
            block_name="decoder",
            batch_normalization=False,
        ):
            with K.name_scope(block_name):
                if activation == "leaky_relu":
                    activation = None
                    conv = LeakyReLU(alpha=0.3)(
                        self.conv_act(
                            inputs, mid_filters, activation, batch_normalization
                        )
                    )
                else:
                    conv = self.conv_act(
                        inputs, mid_filters, activation, batch_normalization
                    )
                conv_tr = Conv2DTranspose(
                    filters=out_filters,
                    activation=None,
                    kernel_size=4,
                    strides=2,
                    padding="same",
                )(conv)
                if batch_normalization:
                    conv_tr = BatchNormalization()(conv_tr)
                conv_tr_act = Activation(activation)(conv_tr)
            return conv_tr_act

        def initialize_resnet_unet_model(self):
            #         print(activation)

            # INPUT
            # shape     - Size of the input images
            # OUTPUT
            # model    - Compiled CNN

            # Define parameters
            max_pooling_size = 2
            max_pooling_strd = 2

            # Load a pretrained ResNet
            num_classes = 2
            resnet50 = ResNet50(
                include_top=False,
                weights="imagenet",
                classes=num_classes,
                input_shape=self.shape,
            )
            resnet50.compile(
                optimizer=Adam(lr=self.lr, amsgrad=self.amsgrad),
                loss="binary_crossentropy",
            )

            # ResNet convolution outputs
            conv5_out = resnet50.get_layer("conv5_block3_out").output
            conv4_out = resnet50.get_layer("conv4_block6_out").output
            conv3_out = resnet50.get_layer("conv3_block4_out").output
            conv2_out = resnet50.get_layer("conv2_block3_out").output

            pool = MaxPooling2D(
                max_pooling_size, strides=max_pooling_strd, padding="same"
            )(resnet50.get_output_at(0))
            dec_center = self.decoder(
                pool,
                mid_filters=self.shape[0] * 2,
                out_filters=self.shape[0],
                activation=self.activation,
                block_name="decoder_center",
            )

            if self.dropout > 0:
                dec_center = Dropout(dropout)(dec_center)

            cat1 = Concatenate()([dec_center, conv5_out])
            dec5 = self.decoder(
                cat1,
                mid_filters=int(self.shape[0] * 2),
                out_filters=int(self.shape[0]),
                activation=self.activation,
                block_name="decoder5",
            )

            if self.dropout > 0:
                dec5 = Dropout(self.dropout)(dec5)

            cat2 = Concatenate()([dec5, conv4_out])
            dec4 = self.decoder(
                cat2,
                mid_filters=int(self.shape[0] * 2),
                out_filters=int(self.shape[0]),
                activation=self.activation,
                block_name="decoder4",
            )

            if self.dropout > 0:
                dec4 = Dropout(self.dropout)(dec4)

            cat3 = Concatenate()([dec4, conv3_out])
            dec3 = self.decoder(
                cat3,
                mid_filters=int(self.shape[0]),
                out_filters=int(self.shape[0] // 4),
                activation=self.activation,
                block_name="decoder3",
            )

            if self.dropout > 0:
                dec3 = Dropout(self.dropout)(dec3)

            cat2 = Concatenate()([dec3, conv2_out])
            dec2 = self.decoder(
                cat2,
                mid_filters=int(self.shape[0] // 2),
                out_filters=int(self.shape[0] // 2),
                activation=self.activation,
                block_name="decoder2",
            )

            if dropout > 0:
                dec2 = Dropout(self.dropout)(dec2)

            dec1 = self.decoder(
                dec2,
                mid_filters=int(self.shape[0] // 2),
                out_filters=int(self.shape[0] // 8),
                activation=self.activation,
                block_name="decoder1",
            )

            if self.dropout > 0:
                dec1 = Dropout(self.dropout)(dec1)

            dec0 = self.conv_act(dec1, out_filters=int(self.shape[0] // 8))
            conv_f = Conv2D(1, 1, activation="sigmoid", padding="same")(dec0)
            flatten_0 = Flatten()(conv_f)
            dense_0 = Dense(
                self.shape[0] / 2,
                kernel_regularizer=l2(1e-6),
                activity_regularizer=l2(1e-6),
            )(flatten_0)
            dropout_0 = Dropout(0.5)(dense_0)
            lk_relu_0 = LeakyReLU(alpha=0.1)(dropout_0)
            dense_1 = Dense(
                2, kernel_regularizer=l2(1e-6), activity_regularizer=l2(1e-6)
            )(lk_relu_0)
            dropout_1 = Dropout(0.5)(dense_1)
            output = Activation("sigmoid")(dropout_1)
            model = Model(inputs=resnet50.get_input_at(0), outputs=output)

            # Compile the model using the binary crossentropy loss and the Adam optimizer for it
            # We used the accuracy as a metric, but F1 score is also a plausible choice
            #             model.compile(
            #                 loss="binary_crossentropy",
            #                 optimizer=Adam(lr=lr, amsgrad=self.amsgrad),
            #                 metrics=["accuracy", recall, f1],
            #             )
            model.compile(
                loss="binary_crossentropy",
                optimizer=Adam(lr=self.lr, amsgrad=self.amsgrad),
                metrics=["accuracy", recall, f1],
            )
            # Print a summary of the model to see what has been generated
            model.summary()

            return model

        def train(
            self,
            epochs,
            steps_per_epoch,
            n_train=85,
            n_val=15,
            batch_size=100,
            data_aug_factor=1,
        ):

            # Early stopping callback after 10 steps
            early_stopping = EarlyStopping(
                monitor="val_loss", min_delta=0, patience=10, verbose=1, mode="auto"
            )

            # Reduce learning rate on plateau after 4 steps
            lr_callback = ReduceLROnPlateau(
                monitor="loss", factor=0.5, patience=4, verbose=1, mode="auto"
            )

            # Save best model
            weights_filename = "model_"
            if self.batch_normalization:
                weights_filename = weights_filename + "batch_"
            weights_filename = (
                weights_filename
                + self.activation
                + "_"
                + str(epochs)
                + "_"
                + "dropout_"
                + str(self.dropout)
                + "_"
                + "{epoch:03d}_"
                + "{f1:03f}_"
                + "{val_accuracy:03f}.h5"
            )
            save_best = ModelCheckpoint(
                weights_filename,
                save_best_only=True,
                monitor="val_loss",
                mode="auto",
                verbose=1,
            )

            # Place the callbacks in a list to be used when training
            #         callbacks = [cb, early_stopping, lr_callback]
            callbacks = [save_best, lr_callback]

            # Train the model using the previously defined functions and callbacks
            history = self.model.fit_generator(
                create_minibatch(
                    X_train, Y_train, data_aug_factor * n_train, batch_size=batch_size
                ),
                steps_per_epoch=steps_per_epoch,
                epochs=epochs,
                use_multiprocessing=False,
                workers=1,
                callbacks=callbacks,
                verbose=1,
                validation_data=create_minibatch(
                    X_val, Y_val, data_aug_factor * n_val, batch_size=batch_size
                ),
                validation_steps=int(steps_per_epoch / 4),
            )

            return history

        def classify(self, X):
            # Subdivide the images into blocks with a stride and patch_size of 16
            img_patches = create_patches(X, 16, 16, padding=24)

            # Predict
            predictions = self.model.predict(img_patches)
            predictions = (predictions[:, 0] < predictions[:, 1]) * 1

            # Predict
            predictions = self.model.predict(img_patches)
            predictions = (predictions[:, 0] < predictions[:, 1]) * 1

            # Regroup patches into images
            return predictions.reshape(X.shape[0], -1)

        def load(self, filename):
            # Load the model (used for submission)
            dependencies = {"recall": recall, "f1": f1}
            self.model = load_model(filename, custom_objects=dependencies)

        def save(self, filename):
            # Save the model (used to then load to submit)
            self.model.save(filename)

In [ ]:
# if not baseline:
if best:
    batch_normalization = True
    activation = "relu"
    dropout = 0
    amsgrad = False
    lr = 1e-4
    noise = 0.1

In [ ]:
# if not baseline:
if best:
    EPOCHS = 80
    STEPS_PER_EPOCH = 100
    batch_size = 100

    model = resnet_unet_model(
        shape=(64, 64, 3),
        batch_normalization=batch_normalization,
        activation=activation,
        dropout=dropout,
        amsgrad=amsgrad,
        lr=lr,
        noise=noise,
    )
    # Train the model
    history = model.train(
        EPOCHS, STEPS_PER_EPOCH, n_train, n_val, batch_size, data_aug_factor
    )

In [ ]:
with open('/trainHistoryDict_best_model', 'wb') as file_pi:
    pickle.dump(history.history, file_pi)

In [ ]:
if not best:
    now_str = str(datetime.now()).replace(" ", "_").replace(".", "_").replace(":", "_")
    if batch_normalization:
        specific_name = now_str + "_batchnorm" + "_dropout{:2.2f}".format(dropout)
    else:
        specific_name = now_str + "_dropout{:2.2f}".format(dropout)
    plt.style.use("ggplot")
    plt.figure(figsize=(15, 10))
    plt.plot(history.history["f1"])
    plt.plot(history.history["val_f1"])
    plt.plot(history.history["loss"])
    plt.plot(history.history["val_loss"])
    plt.title("ResNet-UNet: training vs validation f1 and loss")
    plt.ylabel("value")
    plt.xlabel("epoch")
    plt.legend(
        ["train_f1", "val_f1", "loss", "val_loss"],
        loc="lower left",
        bbox_to_anchor=(0.0, -0.07),
        ncol=4,
        borderaxespad=0,
        frameon=False,
    )
    plt.savefig(specific_name + "_resnet_unet_metrics_f1.png")
    plt.show()

In [ ]:
if best:
    # plot the training loss and accuracy
    plt.style.use("ggplot")
    plt.figure(figsize=(15, 10))
    plt.plot(history.history["loss"][1:], label="train_loss")
    plt.plot(history.history["val_loss"][1:], label="val_loss")
    plt.plot(history.history["accuracy"][1:], label="train_acc")
    plt.plot(history.history["val_accuracy"][1:], label="val_acc")
    plt.title("Training Loss and Accuracy on Dataset")
    plt.xlabel("Epoch #")
    plt.ylabel("Loss/Accuracy")
    plt.legend(loc="lower left")
    plt.savefig("Unet_batchnorm.png")
    plt.show()

In [ ]:
if best:
    from helpers import *

    # from resnet_unet_model import resnet_unet_model

    # Instantiate the model
    model = resnet_unet_model(
        shape=(64, 64, 3),
        batch_normalization=batch_normalization,
        activation=activation,
        dropout=dropout,
        amsgrad=amsgrad,
        lr=lr,
    )

    # Load the model
    model.load("model_batch_relu_80_dropout_0_052_0.849586_0.933300.h5")

    # Print a summary to make sure the correct model is used
    model.model.summary()

    # We add all test images to an array, used later for generating a submission
    image_filenames = []
    for i in range(1, 51):
        image_filename = (
            "data/test_set_images/test_" + str(i) + "/test_" + str(i) + ".png"
        )
        image_filenames.append(image_filename)

    # Set-up submission filename
    submission_filename = "resnet_unet.csv"
    # Generates the submission
    generate_submission(model, submission_filename, *image_filenames)